In [1]:
import os
import sys
import ccxt
import pandas as pd

def update_data(coins):
    btc_price = float(exchange.fetch_ticker('BTC/USDT')['info']['lastPrice'])
    df = pd.DataFrame(columns=['symbol', 'quantity', 'price', 'dollar_value'])
    
    for coin in coins:
        quantity = balance[coin]['total']
        if coin == 'BTC':
            price = btc_price
        else:
            btc_ratio = float(exchange.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
            price = btc_ratio * btc_price
        
        dollar_value = quantity * price
        df = df.append({'symbol': coin,'quantity':quantity,'price':price,'dollar_value':dollar_value}, ignore_index=True)
    
    df['weight'] = list(map(lambda x: x / df['dollar_value'].sum(), df['dollar_value']))
    df = df.sort_values('weight', ascending=False).reset_index(drop=True)
    return df

def rebalance_order(coin1, coin2):
    try:
        exchange.fetch_ticker(coin1 + '/' + coin2)['info']
        return coin1 + '/' + coin2, 'sell'
    except:
        try:
            exchange.fetch_ticker(coin2 + '/' + coin1)['info']
            return coin2 + '/' + coin1, 'buy'
        except:            
            return coin1 + '/BTC', 'sell', coin2 + '/BTC', 'buy' # Note: could coin2 be 'BTC/' + coin/sell?
        
def bnb_order(coin):
    if balance['BNB']['total'] < 1:
        return
    if coin != 'BTC':
        bnb_btc = 2 * float(exchange.fetch_ticker('BNB/BTC')['info']['lastPrice'])
        coin_btc = float(exchange.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
        quantity = bnb_btc / coin_btc
        exchange.create_order(coin + '/BTC', 'market', 'sell', quantity)
        
    exchange.create_order('BNB/BTC', 'market', 'buy', 2)
    print("Purchased 2 BNB with {:0.3f} {}".format(quantity, coin))

In [25]:
file = 'C:/Users/Carter Carlson/Documents/Administrative/api.txt'
with open(file, 'r') as f:
    api = f.readlines()
    apiKey = str(api[0][:len(api[0])-1])
    secret = str(api[1][:len(api[1])])
    
print(apiKey)
print(secret)

nOS2lq1Ki8ALGJyxtUJk5EfHsBPbD25gIYOi3xJNtNMtmSZodWp5Es0WIqsrIQbY
SkrnTWFTOSI4NmixdC8iVq5IDViDyHh7eHcsQyHLJZHe7X0nrdCjuuAkamlbt1jp


In [28]:
file = 'C:/Users/Carter Carlson/Documents/Administrative/api.txt'
with open(file, 'r') as f:
    api = f.readlines()
    apiKey = api[0][:len(api[0])-1]
    secret = api[1][:len(api[1])]
exchange = ccxt.binance({
    'options': {'adjustForTimeDifference': True},
    'apiKey': apiKey,
    'secret': secret}) 
balance = exchange.fetchBalance()
coins = []
[coins.append(asset['asset']) for asset in balance['info']['balances'] if float(asset['free']) > 0]

update_data(coins)

,symbol,quantity,price,dollar_value,weight
0,ETH,4.237491,282.868716,1198.653575,0.204799
1,BTC,0.191365,6257.050000,1197.380561,0.204581
2,XRP,4198.819000,0.280191,1176.470031,0.201009
3,OMG,316.380000,3.635346,1150.150783,0.196512
4,BNB,120.400878,9.386826,1130.182138,0.193100


In [29]:
file = 'C:/Users/Carter Carlson/Documents/Administrative/api.txt'
with open(file, 'r') as f:
    api = f.readlines()
    apiKey = api[0][:len(api[0])-1]
    secret = api[1][:len(api[1])]
exchange = ccxt.binance({
    'options': {'adjustForTimeDifference': True},
    'apiKey': apiKey,
    'secret': secret}) 
balance = exchange.fetchBalance()
coins = []
[coins.append(asset['asset']) for asset in balance['info']['balances'] if float(asset['free']) > 0]

data = update_data(coins)
        
n = 1/len(coins)
thresh = .01
i = 1

while data['weight'][0] - data['weight'][len(data) - 1] > 2 * n * thresh:
    
    order = rebalance_order(data['symbol'][0], data['symbol'][len(data) - 1])
    weight_to_move = (data['weight'][0] - data['weight'][len(data) - 1]) / 2
    quantity = round(weight_to_move * data['dollar_value'].sum() / data[data['symbol'] == order[0][:3]]['price'].values[0], 5)
    print(exchange.create_order(order[0], 'market', order[1], quantity))
    
    if len(order) > 2:
        quantity = round(weight_to_move * data['dollar_value'].sum() / data[data['symbol'] == order[2][:3]]['price'].values[0], 5)
        print(exchange.create_order(order[2], 'market', order[3], quantity))
    
    balance = exchange.fetchBalance()
    data = update_data(coins)


{'info': {'symbol': 'BNBETH', 'orderId': 36579392, 'clientOrderId': 'IZscqZ3yReyUz0Inla85BN', 'transactTime': 1534375391123, 'price': '0.00000000', 'origQty': '3.62000000', 'executedQty': '3.62000000', 'cummulativeQuoteQty': '0.12026450', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'fills': [{'price': '0.03320200', 'qty': '0.37000000', 'commission': '0.00027750', 'commissionAsset': 'BNB', 'tradeId': 8629986}, {'price': '0.03320300', 'qty': '1.01000000', 'commission': '0.00075750', 'commissionAsset': 'BNB', 'tradeId': 8629987}, {'price': '0.03320500', 'qty': '0.01000000', 'commission': '0.00000750', 'commissionAsset': 'BNB', 'tradeId': 8629988}, {'price': '0.03323100', 'qty': '1.10000000', 'commission': '0.00082500', 'commissionAsset': 'BNB', 'tradeId': 8629989}, {'price': '0.03323700', 'qty': '0.36000000', 'commission': '0.00027000', 'commissionAsset': 'BNB', 'tradeId': 8629990}, {'price': '0.03323800', 'qty': '0.77000000', 'commission': '0.00057750', 'co

In [30]:
update_data(coins)

,symbol,quantity,price,dollar_value,weight
0,XRP,4198.819000,0.281152,1180.506746,0.201193
1,BTC,0.187588,6268.720000,1175.935707,0.200414
2,OMG,322.870000,3.642126,1175.933325,0.200413
3,BNB,124.016277,9.420006,1168.234013,0.199101
4,ETH,4.117226,283.427637,1166.935714,0.198880


In [191]:
def update_data(coins):
    df = pd.DataFrame(columns=['symbol', 'quantity', 'price', 'dollar_value'])
    btc_price = float(exchange.fetch_ticker('BTC/USDT')['info']['lastPrice'])
    for coin in coins:
        quantity = balance[coin]['total']
        price = btc_price
        if coin != 'BTC':
            btc_ratio = float(exchange.fetch_ticker(coin + '/BTC')['info']['lastPrice'])
            price *= btc_ratio

        dollar_value = quantity * price
        df = df.append({'symbol': coin,'quantity':quantity,'price':price,'dollar_value':dollar_value}, ignore_index=True)

    df = df.sort_values('dollar_value', ascending=False).reset_index(drop=True)
    df['weight'] = df['dollar_value'].divide(data['dollar_value'].sum())
    return df


def rebalance(coin1, coin2, weight_to_sell, tickers):
    if coin1 + '/' + coin2 in tickers:
        order = coin1 + '/' + coin2, 'sell'
    elif coin2 + '/' + coin1 in tickers:
        order = coin2 + '/' + coin1
    else:
        order = coin1 + '/BTC', 'sell', coin2 + '/BTC', 'buy'

    dollar_amt = abs(weight_to_sell * data['dollar_value'].sum())
    for i in range(0, order, 2):
        coin_amt = float(dollar_amt / data.loc[data['symbol'] == order[i][:3], 'price'])
        single_trade = 'Y'
        if len(order) > 2:
            single_trade = 'N'

        trade_coin(order[i], order[i+1], coin_amt, dollar_amt, fees, single_trade)


def trade_coin(ratio, side, coin_amt, dollar_amt, single_trade):
    exchange.create_order(ratio, 'market', side, coin_amt)
    transactions = pd.read_excel('transactions.xlsx')
    num_rows = len(transactions.index)
    trade_id = transactions['trade_id'][num_rows] + 1
    trade_date = datetime.now()
    fees = dollar_amt * .00075
    transaction = [trade_id, trade_date, side, ratio[:3], ratio[4:], coin_amt, dollar_amt, fees, single_trade]
    for i in range(transaction):
        wb = load_workbook('transactions.xlsx')
        ws = wb.worksheets[0]
        ws.cell(row=num_row+1, column=i+1).value = transaction[i]
        wb.save('transactions.xlsx')


api_file = "C:/Users/Carter Carlson/Documents/Excel References/secret.csv"
api = pd.read_csv(api_file)
exchange = ccxt.binance({'options': {'adjustForTimeDifference': True},'apiKey': api['apiKey'][0],'secret': api['secret'][0]})
balance = exchange.fetchBalance()
wallet = pd.DataFrame(balance['info']['balances'])

coins = []
tickers = set()
[tickers.add(ticker) for ticker in exchange.fetchTickers()]
coins = wallet.loc[wallet['free'].astype(float) > .1, 'asset'].tolist()
data = update_data(coins)
thresh = .01
num_coins = len(data)
avg_weight = 1/num_coins
weight_thresh = avg_weight * thresh

while True:
    heavy_weight_dif = data['weight'][0] - avg_weight
    light_weight_dif = avg_weight - data['weight'][num_coins-1]
    if heavy_weight_dif < weight_thresh and light_weight_dif < weight_thresh:
        break
    elif heavy_weight_dif > light_weight_dif:
        weight_to_sell = light_weight_dif
    else:
        weight_to_sell = heavy_weight_dif

    rebalance(data['symbol'][0], data['symbol'][num_coins-1], weight_to_sell, tickers)
    data = update_data(coins)


b
b
b
b


InsufficientFunds: binance {"code":-2010,"msg":"Account has insufficient balance for requested action."}